<a href="https://colab.research.google.com/github/MajesticFT/SuperAI2/blob/main/Week8_FoodClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!unzip -q '/content/drive/MyDrive/hackathon.zipfile'

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

In [ ]:
train_df =  pd.read_csv('/content/drive/MyDrive/new_train.csv')
train_df.head(5)

,filename,labels
0,000000.png,139
1,000001.png,223 253 256
2,000002.png,121 127 246
3,000003.png,81 282
4,000004.png,2 144 145 205 221 256


In [ ]:
val_df = pd.read_csv('/content/drive/MyDrive/new_val.csv')
val_df.head(5)

,filename,labels
0,009623.png,84 87 133 217
1,009625.png,130 176
2,009626.png,0
3,009627.png,150 201 205 208 221
4,009628.png,215 256


In [ ]:
labels_df = pd.read_csv('/content/drive/MyDrive/new_labels.csv')
labels_df.head(5)

,id,label
0,0,ไม่มี
1,1,กรอบ
2,2,กระดูก
3,3,กระดูกอ่อน
4,4,กระทา


In [ ]:
label_dict = {str(row['id']):row['label'] for i, row in labels_df.iterrows()}

In [ ]:
import matplotlib.pyplot as plt
# freq train df
# plt.figure(figsize=(20,18), dpi=80)
# plt.bar(range(len(label_count)), list(label_count.values()), align='center')
# plt.xticks(range(len(label_count)), list(label_count.keys()))
# plt.show()

In [ ]:
#freq val_df
# plt.figure(figsize=(20,18), dpi=80)
# plt.bar(range(len(val_count)), list(val_count.values()), align='center')
# plt.xticks(range(len(val_count)), list(val_count.keys()))


In [ ]:
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

from sklearn.model_selection import train_test_split
from ast import literal_eval

In [ ]:
train_df['labels_list'] = train_df['labels'].apply(lambda x: [label_dict[label] for label in x.split(" ")])
val_df['labels_list'] = val_df['labels'].apply(lambda x: [label_dict[label] for label in x.split(" ")])




In [ ]:
train_plus_val_df = pd.concat([train_df,val_df], ignore_index=True)
train_plus_val_df.head(5)

,filename,labels,labels_list
0,000000.png,139,[ปู]
1,000001.png,223 253 256,"[อาหารอีสาน, เห็ด, แกง]"
2,000002.png,121 127 246,"[ปลา, ปลาทู, เมี่ยง]"
3,000003.png,81 282,"[ตีน, ไก่]"
4,000004.png,2 144 145 205 221 256,"[กระดูก, ผัก, ผักกาด, หมู, อาหารเหนือ, แกง]"


In [ ]:
x_train = list(train_plus_val_df['filename'])
y_train = list(train_plus_val_df['labels_list'])

#Comment When Summiting To Increase Dataset
x_val = list(val_df['filename'])
y_val = list(val_df['labels_list'])

In [ ]:
#Change labels to binary using MultiLabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer() 
mlb.fit(y_train)

class_key = {}

for (i, label) in enumerate(mlb.classes_):
    class_key[i] = label
    print("{}. {}".format(i, label))


0. กรอบ
1. กระดูก
2. กระดูกอ่อน
3. กระทา
4. กระท้อน
5. กระเฉด
6. กระเทียม
7. กระเพาะปลา
8. กรือโป๊ะ
9. กล้วย
10. กอและ
11. กะทิ
12. กะปิ
13. กะพง
14. กะหรี่
15. กะหล่ำ
16. กะเพรา
17. กากหมู
18. กาแฟ
19. กุนเชียง
20. กุยช่าย
21. กุ้ง
22. ก้อย
23. ก๋วยจั๊บ
24. ก๋วยเตี๋ยว
25. ขจร
26. ขนม
27. ขนมครก
28. ขนมจีน
29. ขนมจีบ
30. ขนมชั้น
31. ขนมตาล
32. ขนมถ้วย
33. ขนมปัง
34. ขนมเบื้อง
35. ขนมเปี๊ยะ
36. ขนุน
37. ขมิ้น
38. ขิง
39. ข้าว
40. ข้าวจี่
41. ข้าวซอย
42. ข้าวต้ม
43. ข้าวต้มมัด
44. ข้าวผัด
45. ข้าวมัน
46. ข้าวมันไก่
47. ข้าวยำ
48. ข้าวราดแกง
49. ข้าวหมก
50. ข้าวหมาก
51. ข้าวเกรียบ
52. ข้าวเหนียว
53. ข้าวโพด
54. ควาย
55. คะน้า
56. คั่ว
57. จิ้น
58. ฉู่ฉี่
59. ชะพลู
60. ชะอม
61. ชา
62. ชาบู
63. ชาเขียว
64. ชีส
65. ช็อกโกแลต
66. ซาลาเปา
67. ซีอิ๊ว
68. ซี่โครง
69. ซุป
70. ซุปเปอร์
71. ซูชิ
72. ดอก
73. ดอง
74. ดอลลี่
75. ดิบ
76. ตะไคร้
77. ตับ
78. ตำ
79. ติ่มซำ
80. ตีน
81. ตือปง
82. ตุ๋น
83. ต้ม
84. ต้มกะทิ
85. ต้มจืด
86. ต้มยำ
87. ต้มยำกุ้ง
88. ต้มส้ม
89. ถั่ว
90. ถั่วฝักยาว
91. ถั่วพู
92. ทร

In [ ]:
# transform the targets of the training and test sets
y_train_bin = mlb.transform(y_train)
y_val_bin = mlb.transform(y_val)

In [ ]:
IMG_SIZE = 512 # Specify height and width of image to match the input format of the model
CHANNELS = 3 # Keep RGB color channels to match the input format of the model

def parse_function(filename, label):
    """Function that returns a tuple of normalized image array and labels array.
    Args:
        filename: string representing path to image
        label: 0/1 one-dimensional array of size N_LABELS
    """
    # Read an image from a file
    image_string = tf.io.read_file('/content/hackathon/'+filename)
    # Decode it into a dense vector
    image_decoded = tf.image.decode_jpeg(image_string, channels=CHANNELS)
    # Resize it to fixed shape
    image= tf.image.resize(image_decoded, [512, 512])
    # Random brightness.
    image = tf.image.random_brightness(image, max_delta=0.5)
    # Random flip left-right
    image = tf.image.random_flip_left_right(image)
    # Random contrast
    image = tf.image.random_contrast(image, lower=0.5, upper=1)
    # Random saturation
    image = tf.image.random_saturation(image, lower=0.5, upper=1)
    # Normalize it from [0, 255] to [0.0, 1.0]
    image_normalized = image / 255.0
    return image_normalized, label
BATCH_SIZE = 64 # Big enough to measure an F1-score
AUTOTUNE = tf.data.experimental.AUTOTUNE # Adapt preprocessing and prefetching dynamically
SHUFFLE_BUFFER_SIZE = 1024 # Shuffle the training data by a chunck of 1024 observations
def create_dataset(filenames, labels, size, is_training=True):
    IMG_SIZE = size
    """Load and parse dataset.
    Args:
        filenames: list of image paths
        labels: numpy array of shape (BATCH_SIZE, N_LABELS)
        is_training: boolean to indicate training mode
    """
    # Create a first dataset of file paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    # Parse and preprocess observations in parallel
    dataset = dataset.map(parse_function, num_parallel_calls=AUTOTUNE)
    if is_training == True:
        # This is a small dataset, only load it once, and keep it in memory.
        # dataset = dataset.cache()
        # Shuffle the data each buffer size
        dataset = dataset.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)
    # Batch the data for multiple steps
    dataset = dataset.batch(BATCH_SIZE)
    # Fetch batches in the background while the model is training.
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    return dataset



In [ ]:
train_ds_1 = create_dataset(x_train, y_train_bin, 512)
val_ds = create_dataset(x_val, y_val_bin)


In [ ]:
for f, l in train_ds_1.take(1):
    print("Shape of features array:", f.numpy().shape)
    print("Shape of labels array:", l.numpy().shape)


Shape of features array: (64, 512, 512, 3)
Shape of labels array: (64, 301)


In [ ]:
@tf.function
def macro_soft_f1(y, y_hat):
    """Compute the macro soft F1-score as a cost (average 1 - soft-F1 across all labels).
    Use probability values instead of binary predictions.
    
    Args:
        y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        
    Returns:
        cost (scalar Tensor): value of the cost function for the batch
    """
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
    macro_cost = tf.reduce_mean(cost) # average on all labels
    return macro_cost


In [ ]:
@tf.function
def macro_f1(y, y_hat, thresh=0.5):
    """Compute the macro F1-score on a batch of observations (average F1 across labels)
    
    Args:
        y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        thresh: probability value above which we predict positive
        
    Returns:
        macro_f1 (scalar Tensor): value of macro F1 for the batch
    """
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1


# Model

In [ ]:
from tensorflow.keras.layers import Flatten, Dense, Activation, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow_hub as hub

In [ ]:
effnet_b7 = 'https://tfhub.dev/google/efficientnet/b7/feature-vector/1' #0.23576 F1 Kaggle 1e-3 LR
effnetv2XL_finetuned = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_xl/feature_vector/2'
effnetv2XL = 'https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_xl/feature_vector/2' #0.1068 val_f1 @epoch 11
bit_r50x1_224 = 'https://tfhub.dev/google/experts/bit/r50x1/in21k/foodstuff/1'


pretrained_model_1 = hub.KerasLayer(effnetv2XL, input_shape=(512, 512, CHANNELS))
pretrained_model_1.trainable = False


classifier_1 = Sequential()
classifier_1.add(pretrained_model_1)
classifier_1.add(Dropout(0.25))
classifier_1.add(Dense(1024))
classifier_1.add(Activation('relu'))
classifier_1.add(Dense(301))
classifier_1.add(Activation('sigmoid'))



In [ ]:
classifier_1.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate= 1e-4),
  loss=macro_soft_f1,
  metrics=[macro_f1])

In [ ]:
classifier_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              207615832 
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1024)              1311744   
                                                                 
 activation (Activation)     (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 301)               308525    
                                                                 
 activation_1 (Activation)   (None, 301)               0         
                                                                 
Total params: 209,236,101
Trainable params: 1,620,269
No

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3, restore_best_weights=True)
history = classifier_1.fit(train_ds_1, 
                         epochs=12,
                         #callbacks = [es],
                         #validation_data=create_dataset(x_val, y_val_bin)
                         )
classifier_1.save('/content/effnetv2XL-Ensemble-NO1-12epochs-512-Augmented-LR1e-4.h5')
!cp /content/effnetv2XL-3Fold-NO1-12epochs-512-Augmented-LR1e-4.h5 /content/drive/MyDrive


Epoch 1/12
161/161 [==============================] - 875s 5s/step - loss: 0.9744 - macro_f1: 0.0301
Epoch 2/12
161/161 [==============================] - 846s 5s/step - loss: 0.9496 - macro_f1: 0.0578
Epoch 3/12
161/161 [==============================] - 847s 5s/step - loss: 0.9302 - macro_f1: 0.0777
Epoch 4/12
161/161 [==============================] - 847s 5s/step - loss: 0.9186 - macro_f1: 0.0892
Epoch 5/12
161/161 [==============================] - 847s 5s/step - loss: 0.9098 - macro_f1: 0.0981
Epoch 6/12
161/161 [==============================] - 848s 5s/step - loss: 0.9042 - macro_f1: 0.1042
Epoch 7/12
161/161 [==============================] - 848s 5s/step - loss: 0.8980 - macro_f1: 0.1102
Epoch 8/12
161/161 [==============================] - 847s 5s/step - loss: 0.8939 - macro_f1: 0.1137
Epoch 9/12
161/161 [==============================] - 847s 5s/step - loss: 0.8899 - macro_f1: 0.1179
Epoch 10/12
161/161 [==============================] - 847s 5s/step - loss: 0.8865 - macro_

#Loading Model (If Needed)

In [ ]:
#Loading Model
model = keras.models.load_model(custom_objects= {'KerasLayer':hub.KerasLayer},filepath='/content/drive/MyDrive/effnetv2XL-3Fold-NO1-12epochs-512-Augmented-LR1e-4.h5', compile=False)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= 1e-4),
  loss=macro_soft_f1,
  metrics=[macro_f1])

In [ ]:
!cp /content/bit_r50x1-Ensemble-NO3-12epochs-224-Augmented-LR1e-4.h5 /content/drive/MyDrive

# Prediction and Submission

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/new_submission.csv')

In [ ]:
test_df.head(10)

,filename,labels
0,011958.png,93 95 229
1,011959.png,35 121 222 256
2,011960.png,121 132 256
3,011963.png,NaN
4,011964.png,NaN
5,011966.png,NaN
6,011967.png,NaN
7,011968.png,NaN
8,011969.png,NaN
9,011970.png,NaN


In [ ]:
dict_conv = {value: key for key, value in label_dict.items()}
dict_conv

{'กรอบ': '1',
 'กระดูก': '2',
 'กระดูกอ่อน': '3',
 'กระทา': '4',
 'กระท้อน': '5',
 'กระเฉด': '6',
 'กระเทียม': '7',
 'กระเพาะปลา': '8',
 'กรือโป๊ะ': '9',
 'กล้วย': '10',
 'กอและ': '11',
 'กะทิ': '12',
 'กะปิ': '13',
 'กะพง': '14',
 'กะหรี่': '15',
 'กะหล่ำ': '16',
 'กะเพรา': '17',
 'กากหมู': '18',
 'กาแฟ': '19',
 'กุนเชียง': '20',
 'กุยช่าย': '21',
 'กุ้ง': '22',
 'ก้อย': '23',
 'ก๋วยจั๊บ': '24',
 'ก๋วยเตี๋ยว': '25',
 'ขจร': '26',
 'ขนม': '35',
 'ขนมครก': '27',
 'ขนมจีน': '28',
 'ขนมจีบ': '29',
 'ขนมชั้น': '30',
 'ขนมตาล': '31',
 'ขนมถ้วย': '32',
 'ขนมถ้วยฟู': '33',
 'ขนมปัง': '34',
 'ขนมเบื้อง': '36',
 'ขนมเปี๊ยะ': '37',
 'ขนุน': '38',
 'ขมิ้น': '39',
 'ขิง': '40',
 'ข้าว': '41',
 'ข้าวจี่': '42',
 'ข้าวซอย': '43',
 'ข้าวต้ม': '44',
 'ข้าวต้มมัด': '302',
 'ข้าวผัด': '45',
 'ข้าวมัน': '46',
 'ข้าวมันไก่': '47',
 'ข้าวยำ': '48',
 'ข้าวราดแกง': '49',
 'ข้าวหมก': '50',
 'ข้าวหมาก': '51',
 'ข้าวเกรียบ': '52',
 'ข้าวเหนียว': '53',
 'ข้าวโพด': '54',
 'ควาย': '55',
 'คะน้า': '56',
 'คั่ว': '5

In [ ]:
def show_prediction(filename, test_df):

    # filename_test = test.loc[test['filename']==filename]['filename'].iloc[0]
    # labels = test.loc[test['label']==label]['label'].iloc[0]
    img_path = os.path.join('/content/hackathon', str(filename))

    # Read and prepare image
    img = cv2.imread(img_path)
    img = img/255.0

    # Generate prediction

    prediction = model.predict(img)                    
    prediction = (prediction > 0.5).astype('int')
    #print(prediction)
    prediction = pd.Series(prediction[0])
    prediction.index = mlb.classes_

    #print(prediction.index)
    if all(element == 0 for element in prediction):
      prediction = ['ไม่มี']
    else: prediction = prediction[prediction>0.5].index.values

    if 'ไม่มี' in prediction and len(prediction)>1:
      prediction = np.delete(prediction, np.where(prediction == 'ไม่มี'))
    #print(prediction)

    # Display image with prediction
    # style.use('default')
    # plt.figure(figsize=(8,4))
    # plt.imshow(Image.open(img_path))
    # plt.title('\n\n{}\n\nPrediction\n{}\n'.format(filename, list(prediction)), fontsize=9)
    # plt.show()

    return prediction

In [ ]:
from tqdm import tqdm
tqdm.pandas()
test_df['labels'][3:] = test_df['filename'][3:].progress_apply(
    lambda x: ' '.join(
       [dict_conv[label] for label in show_prediction(x, test_df)]))


100%|██████████| 3997/3997 [12:08<00:00,  5.49it/s]


In [ ]:
print(sum(test_df['labels'].isnull()))
print(test_df.head(50))
print(test_df.tail(50))

0
      filename                                             labels
0   011958.png                                          93 95 229
1   011959.png                                     35 121 222 256
2   011960.png                                        121 132 256
3   011963.png                 1 10 12 35 108 135 185 218 222 281
4   011964.png                   10 35 36 131 182 185 222 237 281
5   011966.png                                 35 131 179 222 281
6   011967.png                   12 35 32 131 135 179 199 222 281
7   011968.png                                  12 25 131 222 281
8   011969.png                                  35 48 131 222 281
9   011970.png           35 41 48 112 131 135 222 230 281 286 293
10  011971.png                          35 51 131 181 185 222 281
11  011972.png                          35 94 131 135 203 222 281
12  011973.png                          12 35 135 205 214 222 281
13  011974.png                  10 35 131 135 144 205 221 222 281
14  0119

In [ ]:
print(test_df[test_df['labels']=='0'])

In [ ]:
first_test_pic = cv2.imread('/content/hackathon/011996.png')
plt.imshow(cv2.cvtColor(first_test_pic, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
final_csv_path = '/content/submission_week8_Ensemble_Augmented.csv'
test_df.to_csv(final_csv_path, index=False)

In [ ]:
sub = pd.read_csv(final_csv_path)
sub.head(5)

In [ ]:
print(class_key)